<a href="https://colab.research.google.com/github/nehabindle/Yelp-Review-Classification/blob/master/Yelp_Neha_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'Team Drives'/'Data Mining Team'/ gdata

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install contractions

In [0]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Importing Libraries from Keras.


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [5]:
#Checking the version on Tensorflow
tf.__version__


'1.13.1'

In [6]:
#Checking the version of Keras
tf.keras.__version__


'2.2.4-tf'

Load Data


In [0]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('gdata/yelpHotelData.db')
x1 = connection.execute("select * FROM review")

(688329, 10)

In [0]:
description = []
Class = []

data = x1.fetchmany(688329)

for x in data:
  description.append(x[3])
  Class.append(x[8])
trainData = {'Class' : Class, 'Description' : description}
df_X = pd.DataFrame(trainData)

In [12]:
#Convert NR->N , YR->Y
df_X.loc[df_X['Class'] == "NR", 'Class'] = "N"
df_X.loc[df_X['Class'] == "YR", 'Class'] = "Y"
df_X['Class'].value_counts()

N    420785
Y    267544
Name: Class, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
import pickle

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_X['Description'],df_X['Class'], test_size=0.3, random_state=42)

In [15]:
#Checking the shape of Train and Test Data
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(481830,) (206499,)
(481830,) (206499,)


In [16]:
#Checking the length of train and test data
print("Train-set size: ", len(X_train))
print("Test-set size:  ", len(X_test))

Train-set size:  481830
Test-set size:   206499


In [0]:
#Combine into one data-set for some uses below.
data_text = X_train + X_test

In [18]:
#Printing an example to see how data looks like
X_train[1]

'The only place inside the Loop that you can stay for $55/night. Also, the only place you can have a picnic dinner and get a little frisky on the 17th floor roof and then wake up in your room the next morning to an army of ants going in on your picnic leftovers.'

In [19]:
y_train[1]

'N'

Tokenizer
A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set.


In [0]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)


The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.



In [23]:
#Tokenization of train data
%%time
X_train_token = tokenizer.fit_on_texts(X_train)

CPU times: user 1min 5s, sys: 108 ms, total: 1min 5s
Wall time: 1min 6s


In [0]:
pickle_out = open("train_token.pickle","wb")
pickle.dump(X_train_token, pickle_out)
pickle_out.close()

In [25]:
#Tokenization of test data
%%time
X_test_token = tokenizer.fit_on_texts(X_test)
pickle_out_test = open("test_token.pickle","wb")
pickle.dump(X_test_token, pickle_out_test)
pickle_out_test.close()

CPU times: user 30.4 s, sys: 94.7 ms, total: 30.5 s
Wall time: 30.5 s


In [0]:
#set num_words=None above, and then it will automatically be set to the vocabulary-size here.


if num_words is None:
    num_words = len(tokenizer.word_index)

We can then inspect the vocabulary that has been gathered by the tokenizer. This is ordered by the number of occurrences of the words in the data-set. These integer-numbers are called word indices or "tokens" because they uniquely identify each word in the vocabulary.

In [27]:
tokenizer.word_index


{'the': 1,
 'and': 2,
 'a': 3,
 'i': 4,
 'to': 5,
 'of': 6,
 'was': 7,
 'in': 8,
 'is': 9,
 'it': 10,
 'for': 11,
 'that': 12,
 'but': 13,
 'with': 14,
 'you': 15,
 'my': 16,
 'on': 17,
 'this': 18,
 'they': 19,
 'have': 20,
 'we': 21,
 'not': 22,
 'at': 23,
 'so': 24,
 'had': 25,
 'are': 26,
 'were': 27,
 'good': 28,
 'place': 29,
 'as': 30,
 'be': 31,
 'like': 32,
 'here': 33,
 'me': 34,
 'there': 35,
 'out': 36,
 'all': 37,
 'food': 38,
 'if': 39,
 'just': 40,
 'one': 41,
 '\xa0': 42,
 "it's": 43,
 'or': 44,
 'get': 45,
 'up': 46,
 'from': 47,
 'great': 48,
 'very': 49,
 'some': 50,
 '\xa0i': 51,
 'really': 52,
 'their': 53,
 'when': 54,
 'about': 55,
 '\xa0the': 56,
 'an': 57,
 'time': 58,
 'which': 59,
 'can': 60,
 'go': 61,
 'our': 62,
 'your': 63,
 'more': 64,
 'what': 65,
 'would': 66,
 'back': 67,
 'also': 68,
 'service': 69,
 'by': 70,
 'only': 71,
 'too': 72,
 "don't": 73,
 'no': 74,
 'been': 75,
 'nice': 76,
 'little': 77,
 "i'm": 78,
 'other': 79,
 'because': 80,
 'got': 8

We can then use the tokenizer to convert all texts in the training-set to lists of these tokens.

In [0]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)

In [0]:

X_train[1]

'The only place inside the Loop that you can stay for $55/night. Also, the only place you can have a picnic dinner and get a little frisky on the 17th floor roof and then wake up in your room the next morning to an army of ants going in on your picnic leftovers.'

In [0]:
np.array(X_train_tokens[1])

array([   4,  532,   15, 6561,   33,  186,  530,  190,    7,    1,  110,
         57,    4,   24, 1262,   29,   84,  190,    7,   73,  488,    2,
        337, 3245,    5,  963,   36,   23,    1,  960,   25,    4,  235,
          9,   62,   31,   72,    5,  318,   35,  524,  360,  218,  596,
         40,   29,    9,  472, 4889, 2395, 3222,   10,  466,    2,  567,
        603,  691,   43,    8,   11,  194, 4677,   19,   68,   20,  114,
        202,  624,  165,  567,   60, 8087,    8, 3506, 3322,  110,   21,
       1262,    1, 4649, 2841, 1286,   12, 3270, 6521, 1521, 4296, 1696,
         21, 1262,    1, 1359,  551, 2451,   17,    1,  110,  390,   21,
        889,   36,    1,  712,  751,   17,    1,  489,  390, 2799,    1,
       1911,  387,  232,    4,  375,    3,  132,    6,    1,  680,   19,
       4147,  412, 6561,  375,    1,  489,  390,   29,   84,  147,    9,
        166,   40, 3832,   59,  309,  137,  745,    1,    5,   15,  513,
          6,  205,    5,   86,   14,   36,    6,  3

We also need to convert the texts in the test-set to tokens.

In [0]:
X_test_tokens = tokenizer.texts_to_sequences(X_test)

Padding and Truncating Data¶
The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [0]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)

The average number of tokens in a sequence is:

In [0]:
np.mean(num_tokens)

140.80798

The maximum number of tokens in a sequence is:

In [0]:
np.max(num_tokens)

989

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [0]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

369

In [0]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.95387

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [0]:
pad = 'pre'


In [0]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [0]:
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [0]:

X_train_pad.shape

(70000, 369)

The matrix for the test-set has the same shape:

In [0]:
X_test_pad.shape

(30000, 369)

Change of sequence of tokens into padded sequence.Note that when this is input to the Recurrent Neural Network, then it first inputs a lot of zeros. If we had padded 'post' then it would input the integer-tokens first and then a lot of zeros. This may confuse the Recurrent Neural Network.



In [0]:
np.array(X_train_tokens[1])
X_train_pad[1]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

**Tokenizer Inverse Map**

For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))


Helper-function for converting a list of tokens back to a string of words.

In [0]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [0]:
#For example, this is the original text from the data-set:

X_train[1]


'The only place inside the Loop that you can stay for $55/night. Also, the only place you can have a picnic dinner and get a little frisky on the 17th floor roof and then wake up in your room the next morning to an army of ants going in on your picnic leftovers.'

In [0]:
tokens_to_string(X_train_tokens[1])

"i brought my niece here last saturday \xa0it was the first time i had visited as well \xa0it was too cold and almost rainy to hang out at the beach so i thought it would be nice to walk there along park \xa0we arrived just as it started raining adult admission is 7 and kids under 12 get in for free \xa0apparently they also have day every sunday where kids can participate in artist lead first we visited the children's gallery downstairs that featured pin hole photography \xa0then we visited the california seen exhibit on the first floor we checked out the modern art on the second floor \xa0although the collection isn't huge i enjoyed a few of the pieces they displayed \xa0my niece enjoyed the second floor as well since it wasn't just paintings \xa0 i'll definitely add the to my list of things to do with out of town guests when we need to kill an hour or two in the area \xa0it's nothing spectacular but it's a nice little museum with a great view"

**Create the Recurrent Neural Network**


In [0]:
model = Sequential()

In [0]:
embedding_size = 8


In [0]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Adding the gated GRU
model.add(GRU(units=16, return_sequences=True))

In [0]:
#Adding the second GRU with 8 output units
model.add(GRU(units=8, return_sequences=True))

In [0]:
# This code adds the third and final GRU with 4 output units. This will be followed by a dense-layer, so it should only give the final output of the GRU and not a whole sequence of outputs.

model.add(GRU(units=4))

dd a fully-connected / dense layer which computes a value between 0.0 and 1.0 that will be used as the classification output.

In [0]:
model.add(Dense(1, activation='sigmoid'))


In [0]:
#Adam optimizer with the given learning-rate.
optimizer = Adam(lr=1e-3)

In [0]:
#Compile the Keras model so it is ready for training.
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [0]:

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 369, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 369, 16)           1200      
_________________________________________________________________
gru_1 (GRU)                  (None, 369, 8)            600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


**Train the Recurrent Neural Network**

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set.

In [0]:
#print(y_train.loc[y_train['Class'] == "N"])
def mapToBinary(x):
  if x == 'Y':
    return 1
  else:
    return 0
  
y_train_bin = list(map(mapToBinary, y_train))

In [0]:
y_test_bin = list(map(mapToBinary, y_test))

In [0]:
%%time
model.fit(X_train_pad, y_train_bin,
          validation_split=0.05, epochs=1, batch_size=64)

Train on 66500 samples, validate on 3500 samples
Instructions for updating:
Use tf.cast instead.
66500/66500 [==============================] - 1552s 23ms/sample - loss: 0.6627 - acc: 0.6112 - val_loss: 0.6273 - val_acc: 0.6409
CPU times: user 36min 1s, sys: 2min 7s, total: 38min 9s
Wall time: 25min 56s


In [0]:
model.save('gdata/my_model.h5') 

In [0]:
X_test_pad

array([[   0,    0,    0, ...,    4,  102,   22],
       [   0,    0,    0, ...,   17,    1,  696],
       [   0,    0,    0, ...,  473,   32, 1174],
       ...,
       [   0,    0,    0, ...,   57,   35,   73],
       [   0,    0,    0, ...,  475,  151, 2051],
       [   0,    0,    0, ...,    6,  220,  467]], dtype=int32)

In [0]:
# Performance on Test-Set
# Now that the model has been trained we can calculate its classification accuracy on the test-set.

%%time
result = model.evaluate(X_test_pad, y_test_bin)

30000/30000 [==============================] - 553s 18ms/sample - loss: 0.6369 - acc: 0.6298
CPU times: user 14min 25s, sys: 1min 15s, total: 15min 40s
Wall time: 9min 12s


In [0]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 62.98%


In [0]:
predictions = model.predict(X_test_pad)
predictions_bin = list(map(lambda x: 1 if x>=0.5 else 0, predictions))
predictions_bin

from sklearn import metrics

print(metrics.classification_report(y_test_bin, predictions_bin))

              precision    recall  f1-score   support

           0       0.64      0.90      0.75     18123
           1       0.58      0.22      0.32     11877

   micro avg       0.63      0.63      0.63     30000
   macro avg       0.61      0.56      0.53     30000
weighted avg       0.62      0.63      0.58     30000



In [0]:
# #In order to show an example of mis-classified text, we first calculate the predicted sentiment for the first 1000 texts in the test-set.
# %%time
# y_pred = model.predict(x=X_test_pad[0:1000])
# y_pred = y_pred.T[0]

CPU times: user 30.2 s, sys: 2.85 s, total: 33.1 s
Wall time: 19.5 s


In [0]:
# #These predicted numbers fall between 0.0 and 1.0. We use a cutoff / threshold and say that all values above 0.5 are taken to be 1.0 
# #and all values below 0.5 are taken to be 0.0. This gives us a predicted "class" of either 0.0 or 1.0.
# cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [0]:
# #he true "class" for the first 1000 texts in the test-set are needed for comparison.

# cls_true = np.array(y_test[0:1000])

In [0]:
# #We can then get indices for all the texts that were incorrectly classified by comparing all the "classes" of these two arrays
# incorrect = np.where(cls_pred != cls_true)
# incorrect = incorrect[0]

In [0]:
#Of the 1000 texts used, how many were mis-classified?


In [0]:
# y_test_bin = list(map(mapToBinary, y_test))

# predictions = model.predict(X_test_pad)


In [0]:
# predictions_bin = list(map(lambda x: 1 if x>=0.5 else 0, predictions))
# predictions_bin

# from sklearn import metrics

# print(metrics.classification_report(y_test_bin, predictions_bin))

              precision    recall  f1-score   support

           0       0.78      0.92      0.84      2297
           1       0.38      0.17      0.23       703

   micro avg       0.74      0.74      0.74      3000
   macro avg       0.58      0.54      0.54      3000
weighted avg       0.69      0.74      0.70      3000



In [0]:
!ls

sample_data
